<a href="https://colab.research.google.com/github/sudoplox/MCA-Project/blob/master/MCA_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import tweepy
import pandas as pd
from json import dump
from json import *
!pip install wget
import wget
from datetime import date
import numpy as np
import csv

import re
import urllib
import zipfile

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=fce90e1312ae6559d52666e36f732b835dbb318951db3fdcbe7206c17a6b6327
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:


auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth, wait_on_rate_limit=True,
wait_on_rate_limit_notify=True)

In [0]:
def get_existing_images_set(image_path):
    return set(os.listdir(image_path)) #returns set of available downloaded images

In [0]:
def append_list_as_row(file_name, list_of_elem):  
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [0]:
def get_month(s):  # returns month number
    arr = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    for i in range(len(arr)):
        if arr[i]==s:
            return i+1

In [0]:
def get_date(s): # returns date in date type
    datestr = s.split(" ")
    month=get_month(str(datestr[1]))
    date2=int(datestr[2])
    year=int(datestr[-1])
    d = date(year,month,date2)
    return d

In [0]:
def save_tweets(new_tweets,searchquery): # function to save tweets
    media_files=set()
    ids=set()
    t_ids=set()
    rtwts=[]
    user_ids=set()
    tweet=set()
    hashtags = set()
    
    counter1 = 0
    for tweet1 in new_tweets:
        counter1+=1
        try:
            media1 = tweet1.extended_entities.get('media', [])  #get the media link
            if(len(media1) > 0) and media1[0]['type']=="photo":
                x1 = str(media1[0]['media_url']).split('/')
                y1 = x1[-1].split('.')
                id1=y1[0]
                s1=str(tweet1.text)
                if True:
                    stl1 = len(s1.split(" "))
                    a = tweet1._json['created_at']  #time of creation to check for pdip generation
                    d1=get_date(a)
                    hashtags1 = set()
                    g1=tweet1._json['entities']['hashtags'] #hashtags of the image post
                    for h in g1:
                        hashtags1.add(h['text'])
                    t_id1 = tweet1._json['id']
                    r1 = tweet1._json['retweet_count']
                    user_id1=tweet1._json['user']['id_str']
                    results = api.user_timeline(user_id1,count=20)
                    for tweet2 in results:  # get top 20 tweets from an uner profile
                        r2 = tweet2._json['retweet_count']
                        if r1!=r2:
                            try:
                                media2 = tweet2.extended_entities.get('media', [])
                                if(len(media2) > 0) and media2[0]['type']=="photo":  # if the post has an image content or not
                                    x2 = str(media2[0]['media_url']).split('/')
                                    y2 = x2[-1].split('.')
                                    id2=y2[0]
                                    if (id2,id1) not in ids and id1!=id2:
                                        s2=str(tweet2.text)
                                        stl2 = len(s2.split(" "))
                                        if abs(stl1-stl2)<=5:   # the abs diff between the length of caption should be less than 6
                                            if True:
                                                b = tweet2._json['created_at']
                                                d2=get_date(b)
                                                delta = d1-d2
                                                if abs(delta.days)<10:  # to check if the images are posted within 10 days
                                                    hashtags2 = set()
                                                    g2=tweet2._json['entities']['hashtags']
                                                    for h in g2:
                                                        hashtags2.add(h['text'])
                                                    t_id2 = tweet2._json['id']

                                                    intersection = hashtags1.intersection(hashtags2)
                                                    user_id2=tweet2._json['user']['id_str']
                                                    if len(intersection)>0:
                                                        media_files.add((media1[0]['media_url'],media2[0]['media_url']))
                                                        ids.add((id1,id2))
                                                        t_ids.add((t_id1,t_id2))
                                                        rtwts.append((r1,r2))
                                                        user_ids.add((user_id1,user_id2))
                                                        tweet.add((str(tweet1),str(tweet2)))
                                                        break

                            except:
                                pass
        except:
            pass
    inserts = 0
    fin = []
    tweeter = []
    while(True):
        try:
            a=media_files.pop()
            b=ids.pop()
            c=t_ids.pop()
            d=rtwts.pop()
            e=user_ids.pop()
            t = tweet.pop()
            fin.append([a[0],b[0],c[0],d[0],e[0],a[1],b[1],c[1],d[1],e[1]])
            tweeter.append([t[0],t[1]])
            inserts+=1
        except:
            break
    cols = ['media_url_1','media_id_1','tweet_id_1','no_retweets_1','user_id_1',
            'media_url_2','media_id_2','tweet_id_2','no_retweets_2','user_id_2']
    if inserts>0:
        sq_naming = searchquery.split(' ')[0]
        
        existing_files = ['drive/My Drive/Data/Tweets/' + file_t for file_t in os.listdir("drive/My Drive/Data/Tweets/")] #to check if it exists or not
        cur_file1 = "drive/My Drive/Data/Tweets/data_"+sq_naming+".csv"
        cur_file2 = "drive/My Drive/Data/Tweets/tweet_"+sq_naming+".csv"
        
        fin = np.asarray(fin)
        tweeter = np.asarray(tweeter)
        data = pd.DataFrame(fin,columns=cols)
        data2 = pd.DataFrame(tweeter,columns=['tweet_1','tweet_2'])
        if cur_file1 not in existing_files:
            data.to_csv(r'drive/My Drive/Data/Tweets/data_{}.csv'.format(sq_naming),index=False) # if doesnt exists make a new file
            data2.to_csv(r'drive/My Drive/Data/Tweets/tweet_{}.csv'.format(sq_naming),index=False)
        else: # if exists then create a new csv file
            
            data.to_csv(r'drive/My Drive/Data/Tweets/data_{}.csv'.format(sq_naming),mode='a',header=False,index=False)
            data2.to_csv(r'drive/My Drive/Data/Tweets/tweet_{}.csv'.format(sq_naming),mode='a',header=False,index=False)

In [0]:
def call_save_tweets(api,searchQuery,count): # function to get tweets using tweepy in batches of 100
    maxTweets = 2000 # Some arbitrary large number
    tweetsPerQry = 100  # this is the max the API permits
    # fName = 'tweets/'+ # We'll store the tweets in a text file.
    sinceId = None

    # If results only below a specific ID are, set max_id to that ID.
    # else default to no upper limit, start from the most recent tweet matching the search query.
    max_id = -1

    tweetCount = 0
    while tweetCount < maxTweets:
        try:
            if (max_id <= 0):
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry)
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since_id=sinceId)
            else:
                if (not sinceId):
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1))
                else:
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(max_id - 1),
                                            since_id=sinceId)
            if not new_tweets:
                print("No more tweets found")
                break
            save_tweets(new_tweets,searchQuery)
            tweetCount += len(new_tweets)
            print("Downloaded {0} tweets".format(tweetCount))
            max_id = new_tweets[-1].id
        except tweepy.TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break
    print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, searchQuery))

In [0]:
def get_images(hashtag):  #function to download image and save in images folder
    data_files = ["drive/My Drive/Data/Tweets/"+i for i in os.listdir("drive/My Drive/Data/Tweets/") if "data_#" in i]
    existing_images = set(os.listdir("drive/My Drive/Data/Images"))
    for f in data_files:
        ht = re.split('\_|\#|\.csv|,',f)[2]
        if not ht == hashtag[1:]:
            continue
        np_data = np.asarray(pd.read_csv(f))
    #     urlData = urllib.urlopen('https://twitter.com/USABillOfRights/status/468852515409502210/photo/1')
        for i in np_data:
            try:
                fn1 = "drive/My Drive/Data/Images/"+str(i[2])+".jpg"
                if (fn1[27:]+".jpg") not in existing_images:  #check if image1 already exists to not have redownlaods
                    urllib.request.urlretrieve(i[0], fn1)
                    existing_images.add((fn1[27:]+".jpg"))
                fn2 = "drive/My Drive/Data/Images/"+str(i[7])+".jpg"
                if (fn2[27:]+".jpg") not in existing_images:    #check if image2 already exists to not have redownlaods
                    urllib.request.urlretrieve(i[5], (fn2[27:]+".jpg"))
                    existing_images.add(fn2)
                print("got it")
            except:
                pass

In [0]:
# get_images("#graphicdesign")

hmm
life #illustration
hmm
style #illustration
hmm
tgif #illustration
hmm
pandemic #illustration
hmm
inspiration #illustration
hmm
artist #illustration
hmm
nature #illustration
hmm
foodgasm #illustration
hmm
vacation #illustration
hmm
photography #illustration
hmm
likeall #illustration
hmm
likeforlike #illustration
hmm
lockdown #illustration
hmm
millenials #illustration
hmm
COVID #illustration
hmm
artwork #illustration
hmm
photoshoot #illustration
hmm
artistic #illustration
hmm
fashion #illustration
hmm
wow #illustration
hmm
model #illustration
hmm
foodie #illustration
hmm
awesome #illustration
hmm
canon #illustration
hmm
date #illustration
hmm
photographer #illustration
hmm
coronavirus #illustration
hmm
dibujo #illustration
hmm
clubpenguin #illustration
hmm
quarantine #illustration
made it
hmm
ootd #illustration
hmm
artists #illustration
hmm
arte #illustration
hmm
myart #illustration
hmm
covid-19 #illustration


In [0]:
def dataset_generator(search_queries):  #helper function for downloading tweets and images
    for searchQuery in search_queries:
        call_save_tweets(api,searchQuery+" filter:images -filter:retweets min_retweets:100",100)
        get_images(searchQuery)

In [0]:
# s="picturekodak#picturebooks#picturethis#pictures#ballum#bally#ball864#balloons#ballistikboyz妄想#ballers#ballaholic#ballaforcomp#ballaforhead"
# s=s+"#ballbuisting#ballbuster#ballbustingbros#ballbustingsqueeze#ballbustinggrab#ballbustingknee#ballet#ballgutter30åretter#ballhawk#ballistikboyz"
# s=s+"#ballj#balllicking#balloonsb2p#ballondor#ballpython#qball#ballstreching#ballstrechter#ballsacceptedchallange#ballweight#6ballchallenge"
# s=s+"#8ballpool#tinderbestro#tindercancelados#tindergavassier#tinderpriornation#tinderprionation#tinderlimantha#datemyfamily#daterush#dateline"
# s=s+"#datechicago#datedallas#idaten#mealteamsix#mealdersleri#lockdown2020#lockdownhustle#lockdownsouthafrica#lockdownextention#lockdownmzansi#lockdowngraduationday"
# s=s+"#lockdownhouseparty#lockdownart#lockdownactivities#lockdownbirthday#lockdownblues#lockdownbaking#lockdownboredom#lockdownbbq#lockdownbeatchallenge#lockdowncooking"
# s=s+"#lockdownce#lockdowncynicsfail#lockdowncreativity#lockdowndiaries#lockdowndeaths#lockdownday31#lockdownday10#lockdownday11#lockdownend#lockdownextension"
# s=s+"#lockdowneffect#lockdownfun#lockdownguidelines#lockdownhaircut#lockdownindia#lockdownireland#lockdownja#lockdownkamazak#lockdownkedusman"
# s=s+"#lockdownlife#lockdownlarry#lockdownlegends#lockdownlicks#lockdownlifeबचाएगा#lockdownmath#lockdownmemes#lockdownnz#lockdownnow#lockdownnocompromise#lockdownnasionalserentak"
# s=s+"#lockdownondomesticviolence#lockdownonlyoption#lockdownprotest#lockdownproject#lockdownproblems#lockdownpecharcha#lockdownquestions#lockdownradio#lockdownreading"
# s=s+"#lockdownshots#lockdownstories#lockdownsessions#lockdownsaextended#lockdowntheatre#lockdowntrivia#lockdownthrowback#lockdownturnup#lockdownuk#lockdownvvips#lockdownwithindiatv"
# s=s+"#lockdownworship#lockdownwithoutplan#lockdown101#lockdown2#lockdown2special#travel#travelphotography#travelblogger#traveltomorrow#traveling#travelers#travelling#travelcompetition#travelgram#traveloka"
# ss=s.split("#")
# for i in range(len(ss)):
#     ss[i]="#"+ss[i]
# print(ss)


['#picturekodak', '#picturebooks', '#picturethis', '#pictures', '#ballum', '#bally', '#ball864', '#balloons', '#ballistikboyz妄想', '#ballers', '#ballaholic', '#ballaforcomp', '#ballaforhead', '#ballbuisting', '#ballbuster', '#ballbustingbros', '#ballbustingsqueeze', '#ballbustinggrab', '#ballbustingknee', '#ballet', '#ballgutter30åretter', '#ballhawk', '#ballistikboyz', '#ballj', '#balllicking', '#balloonsb2p', '#ballondor', '#ballpython', '#qball', '#ballstreching', '#ballstrechter', '#ballsacceptedchallange', '#ballweight', '#6ballchallenge', '#8ballpool', '#tinderbestro', '#tindercancelados', '#tindergavassier', '#tinderpriornation', '#tinderprionation', '#tinderlimantha', '#datemyfamily', '#daterush', '#dateline', '#datechicago', '#datedallas', '#idaten', '#mealteamsix', '#mealdersleri', '#lockdown2020', '#lockdownhustle', '#lockdownsouthafrica', '#lockdownextention', '#lockdownmzansi', '#lockdowngraduationday', '#lockdownhouseparty', '#lockdownart', '#lockdownactivities', '#lockdow

In [0]:
#@title Default title text
search_queries = ["#model","#supermodel","#sexy","#hot", "#victoriasecret", "#fashion", "#beauty", "#makeup","#tutorial","#vacation",
                  "#qurantine","#bored","#kardashian", "#TBT", "#TGIF", "#nightout", "#porn","#photography", "#photo", "#ugh"]
# all hashtags should have the '#' as well
# the filter:images is added on its own

dataset_generator(search_queries)

Downloaded 45 tweets
Downloaded 81 tweets
Downloaded 106 tweets
No more tweets found
Downloaded 106 tweets, Saved to #model filter:images -filter:retweets min_retweets:10
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got

Rate limit reached. Sleeping for: 45


Downloaded 100 tweets
Downloaded 200 tweets
Downloaded 300 tweets
Downloaded 400 tweets
Downloaded 500 tweets
Downloaded 525 tweets
No more tweets found
Downloaded 525 tweets, Saved to #photo filter:images -filter:retweets min_retweets:10
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got it
got i

In [0]:
# get_existing_images_set()

In [0]:
os.listdir()

['.config', 'sample_data']